In [1]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tnqeet import remove_dots

def benchmark_inference(model, tokenizer, prompt, num_runs=11):
    """Benchmark inference speed"""
    model.eval()
    times = []
    
    print(f"Testing with prompt: '{prompt}'")
    
    with torch.no_grad():
        for i in range(num_runs):
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            input_length = inputs['input_ids'].shape[1]
            
            print(f"Run {i+1}: Input tokens: {input_length}")
            
            start_time = time.time()
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
            end_time = time.time()
            if i == 0:
                continue
            total_length = outputs.shape[1]
            output_length = total_length - input_length  # Only new tokens
            total_time = end_time - start_time
            time_per_token = total_time / output_length if output_length > 0 else float('inf')
            
            # Decode and print the output
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            new_text = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
            
            print(f"  Generated {output_length} tokens in {total_time:.3f}s ({time_per_token:.4f}s/token)")
            print(f"  Full output: '{generated_text}'")
            print(f"  New text: '{new_text}'")
            print(f"  Tokens/sec: {output_length/total_time:.1f}")
            print()
            
            times.append(time_per_token)
    
    return {
        'mean_time_per_token': sum(times) / len(times),
        'std_time_per_token': (sum((t - sum(times)/len(times))**2 for t in times) / len(times))**0.5,
        'tokens_per_second': 1 / (sum(times) / len(times)) if sum(times) > 0 else 0
    }

def create_dotless_model(model_name):
    """Create model with reduced vocabulary"""
    # Load original
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name, 
        torch_dtype=torch.float16, 
        device_map="auto"
    )
    
    # Get dotless vocab
    vocab = tokenizer.get_vocab()
    dotless_vocab = {}
    for token, token_id in vocab.items():
        decoded_token = tokenizer.convert_tokens_to_string([token])
        dotless_token = remove_dots(decoded_token)
        if dotless_token not in dotless_vocab:
            dotless_vocab[dotless_token] = len(dotless_vocab)
    
    print(f"Vocab reduced: {len(vocab)} → {len(dotless_vocab)} ({100*(len(vocab)-len(dotless_vocab))/len(vocab):.1f}% reduction)")
    
    # Resize model
    model.resize_token_embeddings(len(dotless_vocab))
    
    return model, tokenizer

# Test with a smaller model first
model_name = "/hdd/shared_models/ALLaM-7B-Instruct-preview"  # Start with smaller model
prompt = "مرحبا كيف حالك"  # Arabic text

print("Benchmarking original model...")
tokenizer_orig = AutoTokenizer.from_pretrained(model_name)
model_orig = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

orig_results = benchmark_inference(model_orig, tokenizer_orig, prompt)
print(f"Original: {orig_results['mean_time_per_token']:.4f}±{orig_results['std_time_per_token']:.4f}s/token")
print(f"Original: {orig_results['tokens_per_second']:.1f} tokens/sec")


print("\nBenchmarking dotless model...")
model_dotless, tokenizer_dotless = create_dotless_model(model_name)

dotless_results = benchmark_inference(model_dotless, tokenizer_dotless, prompt)
print(f"Dotless: {dotless_results['mean_time_per_token']:.4f}±{dotless_results['std_time_per_token']:.4f}s/token")
print(f"Dotless: {dotless_results['tokens_per_second']:.1f} tokens/sec")

# Calculate speedup
speedup = orig_results['mean_time_per_token'] / dotless_results['mean_time_per_token']
throughput_improvement = dotless_results['tokens_per_second'] / orig_results['tokens_per_second']
print(f"\nSpeedup: {speedup:.2f}x ({100*(speedup-1):.1f}% faster per token)")
print(f"Throughput improvement: {throughput_improvement:.2f}x ({100*(throughput_improvement-1):.1f}% more tokens/sec)")

# Memory usage comparison
print(f"\nMemory usage:")
print(f"Original model: {sum(p.numel() for p in model_orig.parameters() if hasattr(locals().get('model_orig'), 'parameters')) / 1e6:.1f}M params")
print(f"Dotless model: {sum(p.numel() for p in model_dotless.parameters()) / 1e6:.1f}M params")

# Clear memory
del model_orig
torch.cuda.empty_cache()

Benchmarking original model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Testing with prompt: 'مرحبا كيف حالك'
Run 1: Input tokens: 3
Run 2: Input tokens: 3
  Generated 4 tokens in 0.133s (0.0334s/token)
  Full output: 'مرحبا كيف حالك اليوم؟ '
  New text: 'اليوم؟ '
  Tokens/sec: 30.0

Run 3: Input tokens: 3
  Generated 4 tokens in 0.133s (0.0334s/token)
  Full output: 'مرحبا كيف حالك اليوم؟ '
  New text: 'اليوم؟ '
  Tokens/sec: 30.0

Run 4: Input tokens: 3
  Generated 4 tokens in 0.136s (0.0340s/token)
  Full output: 'مرحبا كيف حالك اليوم؟ '
  New text: 'اليوم؟ '
  Tokens/sec: 29.4

Run 5: Input tokens: 3
  Generated 4 tokens in 0.134s (0.0335s/token)
  Full output: 'مرحبا كيف حالك اليوم؟ '
  New text: 'اليوم؟ '
  Tokens/sec: 29.9

Run 6: Input tokens: 3
  Generated 4 tokens in 0.133s (0.0332s/token)
  Full output: 'مرحبا كيف حالك اليوم؟ '
  New text: 'اليوم؟ '
  Tokens/sec: 30.2

Run 7: Input tokens: 3
  Generated 4 tokens in 0.135s (0.0338s/token)
  Full output: 'مرحبا كيف حالك اليوم؟ '
  New text: 'اليوم؟ '
  Tokens/sec: 29.6

Run 8: Input tokens: 3
  Ge

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Vocab reduced: 64000 → 49983 (21.9% reduction)
Testing with prompt: 'مرحبا كيف حالك'
Run 1: Input tokens: 3
Run 2: Input tokens: 3
  Generated 50 tokens in 1.652s (0.0330s/token)
  Full output: 'مرحبا كيف حالك اليوم ؟ أنا بخير شكراً لك . هل يمكنني أن أسألك سؤالاً شخصياً إذا لم يكن ذلك يزعج كثيراً ؟ بالطبع لا يزعجني على الإطلاق . ما هو السؤال الذي يدور في ذهنك الآن ؟ شكراً لك على ثقتك بي . أنا هنا'
  New text: 'اليوم ؟ أنا بخير شكراً لك . هل يمكنني أن أسألك سؤالاً شخصياً إذا لم يكن ذلك يزعج كثيراً ؟ بالطبع لا يزعجني على الإطلاق . ما هو السؤال الذي يدور في ذهنك الآن ؟ شكراً لك على ثقتك بي . أنا هنا'
  Tokens/sec: 30.3

Run 3: Input tokens: 3
  Generated 50 tokens in 1.675s (0.0335s/token)
  Full output: 'مرحبا كيف حالك اليوم ؟ أنا بخير شكراً لك . هل يمكنني أن أسألك سؤالاً شخصياً إذا لم يكن ذلك يزعج كثيراً ؟ بالطبع لا يزعجني على الإطلاق . ما هو السؤال الذي يدور في ذهنك الآن ؟ شكراً لك على ثقتك بي . أنا هنا'
  New text: 'اليوم ؟ أنا بخير شكراً لك . هل يمكنني أن أسألك سؤالاً شخصياً إذا لم ي